In [10]:
# warnings....ignore
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd

pd.options.mode.chained_assignment = None

In [11]:
DATA = []
BUCKETS = [0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 2]
TIME_VECTOR = 5  # second

In [12]:
import json


def get_data_from_file(filename: str):
    f = open(filename, "r")
    while True:
        line = f.readline()
        if not line:
            break
        # Standardize json to dict
        row = json.loads(line)
        # Append to Array
        DATA.append(row)

In [13]:
get_data_from_file("../statics/request_data_with_anomaly_pod_instance.json")

In [14]:
group_by_el = ["path", "method", "service_name", "controller_pod"]

In [15]:
# Create dataframe
root_df = pd.DataFrame(DATA)

In [16]:
root_df.head()

,timestamp,path,method,service_name,controller_pod,duration_time
0,"31/01/2024, 11:01:56",/home,GET,process_log,pod-1,0.242
1,"31/01/2024, 11:01:57",/users,GET,process_log,pod-2,0.439
2,"31/01/2024, 11:01:57",/home,GET,process_log,pod-1,0.459
3,"31/01/2024, 11:01:58",/home,GET,process_log,pod-1,0.252
4,"31/01/2024, 11:01:58",/users,GET,process_log,pod-2,0.527


In [101]:

arr_data = []


# BUCKETS = [0.005, 0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 2]
def bucketing(row_data: pd.Series):
    duration_time = row_data["duration_time"]
    for bucket in BUCKETS:
        if duration_time <= bucket:
            row = {
                "timestamp": row_data.timestamp,
                "path": row_data.path,
                "method": row_data.method,
                "service_name": row_data.service_name,
                "controller_pod": row_data.controller_pod,
                "duration_time": duration_time,
                "le": str(bucket)
            }
            arr_data.append(row)
    # Always append bucket +Inf
    arr_data.append({
        "timestamp": row_data.timestamp,
        "path": row_data.path,
        "method": row_data.method,
        "service_name": row_data.service_name,
        "duration_time": duration_time,
        "controller_pod": row_data.controller_pod,
        "le": "+Inf"
    })
    # print(arr_data)


root_df.apply(lambda x: bucketing(x), axis=1)
df_with_bucketed = pd.DataFrame.from_records(arr_data)
df_with_bucketed

,timestamp,path,method,service_name,controller_pod,duration_time,le
0,"31/01/2024, 11:01:56",/home,GET,process_log,pod-1,0.242,0.25
1,"31/01/2024, 11:01:56",/home,GET,process_log,pod-1,0.242,0.5
2,"31/01/2024, 11:01:56",/home,GET,process_log,pod-1,0.242,1
3,"31/01/2024, 11:01:56",/home,GET,process_log,pod-1,0.242,2
4,"31/01/2024, 11:01:56",/home,GET,process_log,pod-1,0.242,+Inf
...,...,...,...,...,...,...,...
60604,"31/01/2024, 12:42:48",/users,GET,process_log,pod-1,0.417,+Inf
60605,"31/01/2024, 12:42:49",/home,GET,process_log,pod-2,0.465,0.5
60606,"31/01/2024, 12:42:49",/home,GET,process_log,pod-2,0.465,1
60607,"31/01/2024, 12:42:49",/home,GET,process_log,pod-2,0.465,2


In [102]:
from datetime import datetime

df_with_bucketed['timestamp'] = df_with_bucketed['timestamp'].apply(
    lambda x: datetime.strptime(x, "%d/%m/%Y, %H:%M:%S"))
# Convert to datetime pandas and set datetime columns is index for dataframe
df_with_bucketed['timestamp'] = pd.to_datetime(root_df['timestamp'])
df_with_bucketed = df_with_bucketed.set_index('timestamp')

C:\Users\vunv\AppData\Local\Temp\ipykernel_7468\3785656374.py:5: UserWarning: Parsing dates in %d/%m/%Y, %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_with_bucketed['timestamp'] = pd.to_datetime(root_df['timestamp'])


In [104]:
df_with_bucketed.query("path == '/home' and controller_pod == 'pod-1' and le == '+Inf'")

,path,method,service_name,controller_pod,duration_time,le
timestamp,,,,,,
2024-01-31 11:01:58,/home,GET,process_log,pod-1,0.242,+Inf
2024-01-31 11:02:01,/home,GET,process_log,pod-1,0.459,+Inf
2024-01-31 11:02:02,/home,GET,process_log,pod-1,0.252,+Inf
2024-01-31 11:02:05,/home,GET,process_log,pod-1,0.183,+Inf
2024-01-31 11:02:08,/home,GET,process_log,pod-1,0.658,+Inf
...,...,...,...,...,...,...
NaT,/home,GET,process_log,pod-1,0.102,+Inf
NaT,/home,GET,process_log,pod-1,0.722,+Inf
NaT,/home,GET,process_log,pod-1,0.354,+Inf


In [116]:
df_with_bucketed['count'] = df_with_bucketed.groupby(
    ['path', 'method', 'controller_pod', 'service_name', 'le']).cumcount() + 1
df_with_bucketed.groupby([pd.Grouper(freq="30s"), 'path', 'method', 'controller_pod', 'service_name', 'le']).agg(
    {'count': 'last'}).reset_index()[['timestamp', 'path', 'method', 'controller_pod', 'service_name', 'le', 'count']].to_csv('out.csv')